In [1]:
import os
import ogr
import gdal
import zipfile
import numpy as np
from tqdm import tqdm
from uuid import uuid4
import geopandas as gpd
from datetime import date, timedelta
from sentinelsat import SentinelAPI
from shapely.geometry import Polygon
from collections import OrderedDict
from shapely.wkt import loads
import shapely
shapely.speedups.disable()


### Select area

In [2]:
area_name = 'Dion'

if area_name == 'Dion':
    shp_name = 'DionSimpleBoundary.shp' #  
    tiles = None
if area_name == 'Hymettus':
    shp_name = 'HymettusSimpleBoundary.shp'
    tiles = ['T34SGG']
if area_name == 'Samaria':
    shp_name =  'SamariaSimpleBoundary.shp'   # SamariaSiteBoundary.shp too long wkt, not accepted for API query
    tiles = ['T34SGE']

# Settings

* Set project directory:

In [3]:
projectdir = r'C:\Users\csoti\Desktop\notebooks\AUTH'

* Set Project settings  

In [1]:
project = {} 
project['inputs'] = os.path.join(projectdir, 'inputs')
if not os.path.exists(project['inputs']):
    os.mkdir(project['inputs'])

# where shapefiles are stored
project['shp'] = os.path.join(projectdir,'inputs',shp_name)                    # shapefile path
assert os.path.exists(project['shp'])

project['raw_images'] = os.path.join(projectdir, area_name, 'raw_images')      # where downloaded images are stored
if not os.path.exists(project['raw_images']):
    os.mkdir(project['raw_images'])
    
project['sub_images'] = os.path.join(projectdir, area_name, 'sub_images')      # where final cropped products are stored
if not os.path.exists(project['sub_images']):
    os.mkdir(project['sub_images'])

* Set Sentinel settings

In [5]:
sentinel={}
sentinel['api'] = SentinelAPI('christiansot', 'belit61218', 'https://scihub.copernicus.eu/dhus') # username, password
sentinel['platformname'] = 'Sentinel-2'                                                          # product selection for download
sentinel['processinglevel'] = 'Level-2A'                                                         # processing level of product for download
# output product settings
sentinel['tiles'] = tiles                                                  # or, None
sentinel['bands'] = ['B02_10m', 'SCL_20m', 'WVP_60m']                      # band selection as <band_resolution>
sentinel['EPSG'] = 'EPSG:32634'                                            # reference system of final product
sentinel['no_data'] = -9999                                                # no data value for final product

* Set Time settings

In [7]:
# Set time period: 
WEEKLY = 4
end = date.today()                      # or, end = date(2020, 11, 19) 
start = end - timedelta(days=WEEKLY)
#
end_date = end.strftime("%Y%m%d"); start_date = start.strftime("%Y%m%d")
sentinel['date'] = (start_date,end_date)
sentinel['date']

('20210219', '20210223')

# Functions

In [8]:
def download_S2(rawdir, api, area, date, platformname, processinglevel, tiles):
    download_candidate = api.query(area=area, date=date,platformname= platformname, processinglevel = processinglevel) 
    download_products = OrderedDict()
    downls_list = []
    tiles = sentinel['tiles']
    title_found_sum = 0

    for key, value in download_candidate.items():
        for k, v in value.items():

            if tiles != None:
                for tile in tiles:
                    if k == 'title' and v.split('_')[5]== tile:
                        prod_title = v
                        downls_list.append(os.path.join(rawdir, prod_title + '.zip'))
                        title_found_sum += 1
                        download_products[key] = value
                        for prodkey, size in value.items():
                            if prodkey == 'size' :
                                print("title: " + prod_title + " | " + size)
            if tiles == None:
                if k == 'title':
                    prod_title = v
                    downls_list.append(os.path.join(rawdir, prod_title + '.zip'))
                    title_found_sum += 1
                    download_products[key] = value
                elif k == 'size':
                    print("title: " + prod_title + " | " + v)

    print("Total found " + str(title_found_sum) +
      " title of " + str(api.get_products_size(download_products)) + " GB")

    rawdir = project['raw_images']
    os.chdir(rawdir)
    if (rawdir + "*.zip") not in [value for value in download_products.items()]:
        api.download_all(download_products)
        print("Finish Downloading")
    else:
        print("Escaping download")

    return downls_list


def un_zipFiles(downls_list):
    """
    Unzip files. Requires folder path
    """    
    for file in tqdm(downls_list):
        if file.endswith('.zip'):
            print('\n')
            print("unzip: " + os.path.basename(file))
            path = os.path.dirname(file)
            zip_file = zipfile.ZipFile(file)
            for names in zip_file.namelist():
                zip_file.extract(names,path)
            zip_file.close() 
            print('Finish Unzipping')
                        

def find(rootdir, endswith):
    """
    Find selected data from a rootdir that end with: ex. '.tif'
    """    
    finder = []
    for root, dirs, files in os.walk(rootdir):
        for f in files:
            if f.endswith(endswith):
                fullpath = os.path.join(root, f)
                finder.append(fullpath)
    return finder
        

def wkt_from_WGS84_shp(shp):
    """
    Get wkt from WGS84 shapefile. Requires shapefile path
    """    
    shp_input = ogr.Open(shp)
    layer = shp_input.GetLayer()
    shp_prj = layer.GetSpatialRef().ExportToProj4()
    # check if shp in in WGS84 ref. system
    if shp_prj == '+proj=longlat +datum=WGS84 +no_defs':

        geoms = []
        for feature in layer:
            geom = feature.GetGeometryRef()
            geoms.append(geom.ExportToWkt())

        pol = loads(geoms[0])
        wkt = Polygon(pol.exterior.coords).wkt
        return wkt
    else:
        raise ValueError('shapefile not in WGS84 ref. system')
                

def reprj_shp(shp, EPSG):
    """
    Reproject shapefile to ESPG. Requires shapefile path and EPSG as ''EPSG:32634''
    """  
    outdir = os.path.dirname(shp) #  reprojects to the same folder where input shapefile is
    input_shp = gpd.read_file(shp)
    # change CRS 
    output_shp = input_shp.to_crs(EPSG)
    # write shp file
    epsg = '_' + 'epsg' + EPSG.split(':')[1]
    reprj_shp_name = os.path.basename(shp).replace('.shp', epsg + '.shp')
    reprj_shp_path = os.path.join(outdir,reprj_shp_name)
    if not os.path.exists(reprj_shp_path):
        reprj_shp = output_shp.to_file(reprj_shp_path)
    return reprj_shp_path


def get_coords_from_shp(shp):
    """
    Get shapefile coordinates. Requires shapefile path 
    """  
    ds = ogr.Open(shp)
    lyr = ds.GetLayer()
    ft = lyr.GetFeature(0)
    geom = ft.GetGeometryRef()
    coords_shp = geom.GetEnvelope()
    geom = None
    ft = None
    lyr = None
    ds = None
    return coords_shp


def subset_tif(outdir, tif_list, shp, extent, dstNodata):
    """
    Subset images to shapefile extent. 
    Requires; output path, list of images-path, shapefile path/extent, and no-data value
    """  
    for file in tif_list:
        if os.path.exists(file):
            filename = os.path.basename(file).replace('.jp2', '_sub.tif')
            outtif = os.path.join(outdir,filename)

            if not os.path.exists(outtif):
                
                input_tif = gdal.Open(file)
                options = gdal.WarpOptions(cutlineDSName=shp,outputBounds= extent, dstNodata=dstNodata)
                outBand = gdal.Warp(srcDSOrSrcDSTab=input_tif,
                    destNameOrDestDS=outtif,
                    options=options)                
                
                input_tif = None

                print('File subsetting to AOI:')
                print(outtif)
            
            else:
                print('File already exists:')
                print(outtif)
                
                
def download_subset_S2bands(sentinel_dict, project_dict):
    """
    Download/subset sentinel images to shapefile.
    Requires; sentinel settings dict, project settings dict
    """      
    # assert shp in WGS84 ref. system and get wkt
    sentinel_dict['area'] = wkt_from_WGS84_shp(project_dict['shp'])
    
    # download s2 files for given time range and aoi
    downls_list = download_S2(rawdir=          project_dict['raw_images'],
                                api=             sentinel_dict['api'], 
                                area=            sentinel_dict['area'], 
                                date=            sentinel_dict['date'], 
                                platformname=    sentinel_dict['platformname'], 
                                processinglevel= sentinel_dict['processinglevel'], 
                                tiles =          sentinel_dict['tiles'])
    
    un_zipFiles(downls_list)
    
    # reprj shp to desired EPSG and get extent
    reprjShp = reprj_shp(project_dict['shp'], sentinel_dict['EPSG'])
    coords_shp = get_coords_from_shp(reprjShp)
    extent = [coords_shp[0],coords_shp[2],coords_shp[1],coords_shp[3]]

    # get bands
    bands_list = sentinel_dict['bands']
    for band in bands_list:
        for file in downls_list:
            foldername = file.replace('.zip','.SAFE')
            folderdir = os.path.join(project['raw_images'], foldername)
            band_list = find(folderdir, band + '.jp2')
            if len(band_list) == 0:
                raise ValueError('Band %s does not exists in Sentinel-2 products.', band)

        # Subset bands to shp extent
            tif_aoi =  subset_tif(outdir =  project_dict['sub_images'],
                                tif_list =  band_list,
                                     shp =    reprjShp, 
                                  extent =    extent, 
                               dstNodata = sentinel_dict['no_data'])

# Run

In [9]:
download_subset_S2bands(sentinel_dict= sentinel, project_dict= project)

title: S2B_MSIL2A_20210222T092029_N0214_R093_T34TFK_20210222T120056 | 999.29 MB
Total found 1 title of 0.98 GB


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.05G/1.05G [02:54<00:00, 6.02MB/s]
MD5 checksumming: 100%|███████████████████████████████████████████████████████████| 1.05G/1.05G [00:14<00:00, 73.7MB/s]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Finish Downloading


unzip: S2B_MSIL2A_20210222T092029_N0214_R093_T34TFK_20210222T120056.zip


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.19s/it]

Finish Unzipping


File subsetting to AOI:
C:\Users\csoti\Desktop\notebooks\AUTH\Dion\sub_images\T34TFK_20210222T092029_B02_10m_sub.tif
File subsetting to AOI:
C:\Users\csoti\Desktop\notebooks\AUTH\Dion\sub_images\T34TFK_20210222T092029_SCL_20m_sub.tif
File subsetting to AOI:
C:\Users\csoti\Desktop\notebooks\AUTH\Dion\sub_images\T34TFK_20210222T092029_WVP_60m_sub.tif
